<a href="https://colab.research.google.com/github/zahrasalehi1380/zahra2025/blob/main/examMachineL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# کلون کردن دیتاست و استخراج فایل‌ها
!git clone https://github.com/F-Aghaeipoor/Knee-X-ray.git
!unrar x -o+ "/content/Knee-X-ray/train.rar" "/content/Knee-X-ray/"
!unrar x -o+ "/content/Knee-X-ray/test.rar" "/content/Knee-X-ray/"

fatal: destination path 'Knee-X-ray' already exists and is not an empty directory.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/Knee-X-ray/train.rar

Extracting  /content/Knee-X-ray/train/AbNormal/OP130.jpg                   0%  OK 
Extracting  /content/Knee-X-ray/train/AbNormal/OP130_aug_0.jpeg            0%  OK 
Extracting  /content/Knee-X-ray/train/AbNormal/OP131.jpg                   0%  OK 
Extracting  /content/Knee-X-ray/train/AbNormal/OP131_aug_0.jpeg            0%  OK 
Extracting  /content/Knee-X-ray/train/AbNormal/OP132.jpg                   0%  OK 
Extracting  /content/Knee-X-ray/train/AbNormal/OP132_aug_0.jpeg            0%  OK 
Extracting  /content/Knee-X-ray/train/AbNormal/OP133.jpg                   0%  OK 
Extracting  /content/Knee-X-ray/train/AbNormal/OP133_aug_0.jpeg            0%  OK 
Extracting  /content/Knee-X-ray/train/AbNormal/OP134.jpg  

In [22]:
# تعریف مسیرها
import os
train_dir = "/content/Knee-X-ray/train"
test_dir = "/content/Knee-X-ray/test"

In [23]:
# بررسی ساختار فایل‌ها
for root, dirs, files in os.walk(train_dir):
    print(root, "has", len(files), "files")

/content/Knee-X-ray/train has 0 files
/content/Knee-X-ray/train/AbNormal has 2520 files
/content/Knee-X-ray/train/Normal has 1260 files


## **Section A : Pre-processing**
<div dir="rtl" style="text-align: right;">

**پیش‌ پردازش و حذف نویز**

خط لوله پیشنهادی:

*   تبدیل به تصویر خاکستری (Grayscale)
*   نرمال‌سازی شدت پیکسل‌ها به بازه [0,1]
*   فیلتر گوسی (Gaussian Filter) برای حذف نویز
*   همسان‌سازی هیستوگرام (CLAHE) برای بهبود کنتراست محلی
*   تغییر اندازه (Resize) برای یکسان‌سازی ورودی مدل
<div>

In [24]:
# پیش‌پردازش تصویر
import cv2
import numpy as np

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    normalized = gray / 255.0
    blurred = cv2.GaussianBlur(normalized, (5, 5), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply((blurred * 255).astype('uint8'))
    resized = cv2.resize(enhanced, (224, 224))
    return resized

<div dir="rtl" style="text-align: right;">

# **افزایش داده (Data Augmentation)**
در این پروژه با داده‌های پزشکی (اشعه ایکس) مواجهیم، بنابراین تکنیک‌ها باید بدون تحریف ساختار استخوان انجام شود.

 تکنیک‌های مناسب:
1.   چرخش جزئی (Rotation ±10°):
تصاویر اشعه ایکس ممکن است با زاویه‌های کمی گرفته شده باشند. این تکنیک به مدل یاد می‌دهد نسبت به زاویه چرخش مقاوم باشد.
2.   جابجایی (Translation): کمی جابجایی افقی یا عمودی ساختار را حفظ می‌کند اما تنوع در داده ایجاد می‌کند.
1.   تغییر روشنایی/کنتراست (Brightness/Contrast): با توجه به تنوع در کیفیت دستگاه‌های تصویربرداری، این تکنیک به افزایش مقاومت مدل در برابر اختلافات تصویری کمک می‌کند.
<div>

In [25]:
from torchvision import transforms

augmentations = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.05, 0.05)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
])

In [26]:
# استخراج ویژگی‌ها برای مدل‌های ML
from skimage.feature import hog

def extract_features(img):
    return hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2), feature_vector=True)

# بارگذاری تصاویر و برچسب‌ها

from tqdm import tqdm
X, y = [], []

for label, folder in enumerate(["Normal", "AbNormal"]):
    folder_path = os.path.join(train_dir, folder)
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        continue
    for filename in tqdm(os.listdir(folder_path)):
        path = os.path.join(folder_path, filename)
        img = preprocess_image(path)
        features = extract_features(img)
        X.append(features)
        y.append(label)

100%|██████████| 2520/2520 [00:22<00:00, 112.25it/s]


<div dir="rtl" style="text-align: right;">
دو مدل یادگیری ماشین سنتی
مدل‌ها:

1.  Support Vector Machine (SVM)
2.  Random Forest (RF)

این دو مدل برای مسائل با تعداد داده کم و ویژگی‌های استخراج‌شده مناسب‌اند، و با تنظیمات مناسب عملکرد خوبی دارند.

پیش‌نیاز: استخراج ویژگی از تصاویر
چون مدل‌های ML نمی‌توانند مستقیم تصاویر خام را پردازش کنند، باید ویژگی‌هایی استخراج شوند (مثلاً با HOG یا PCA).

> دلیل انتخاب مدل‌ها:

SVM مناسب داده‌های کم و تفکیک‌پذیر

Random Forest مقاوم به نویز و دارای توانایی مدیریت داده نامتوازن با class_weight
<div>

In [27]:
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)
# مدل RF
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced')
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("\nRandom Forest:\n", classification_report(y_test, y_pred_rf))


Random Forest:
               precision    recall  f1-score   support

           0       0.89      0.60      0.72       252
           1       0.83      0.96      0.89       504

    accuracy                           0.84       756
   macro avg       0.86      0.78      0.80       756
weighted avg       0.85      0.84      0.83       756



In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# دیتاست سفارشی PyTorch
class KneeXrayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        for label, cls in enumerate(["Normal", "AbNormal"]):
            folder = os.path.join(root_dir, cls)
            if not os.path.exists(folder):
                print(f"Folder not found: {folder}")
                continue
            for fname in os.listdir(folder):
                self.samples.append((os.path.join(folder, fname), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert("L")
        if self.transform:
            img = self.transform(img)
        return img, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
train_dataset = KneeXrayDataset(train_dir, transform=transform)
test_dataset = KneeXrayDataset(test_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# معماری CNN ساده
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 54 * 54, 128)
        self.fc2 = nn.Linear(128, 2)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 54 * 54)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
for epoch in range(5):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()



In [17]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

print("\nSimple CNN:\n", classification_report(all_labels, all_preds))


Simple CNN:
               precision    recall  f1-score   support

           0       0.80      0.62      0.70       180
           1       0.83      0.92      0.87       360

    accuracy                           0.82       540
   macro avg       0.81      0.77      0.79       540
weighted avg       0.82      0.82      0.82       540



 <div dir="rtl" style="text-align: right;">

# **یادگیری انتقالی (Transfer Learning)**

مدل پیشنهادی: ResNet18 یا EfficientNet-B0
هردو مدل کم‌حجم با دقت بالا هستند و برای داده‌های پزشکی پرکاربردند.

Freeze لایه‌های اولیه و فقط fine-tune کردن لایه‌های آخر به مدل کمک می‌کند روی داده جدید بیش‌برازش نکند.

 <div dir="rtl" style="text-align: right;">

# **استراتژی مقابله با بیش‌برازش**


1.   Regularization و Dropout

  در مدل CNN از Dropout(0.3) برای کاهش وابستگی به نورون‌های خاص استفاده شد.
2.   Early Stopping

  متوقف کردن آموزش در صورتی که loss اعتبارسنجی برای چند epoch ثابت بماند.
1.   Data Augmentation

  استفاده از چرخش، روشنایی، و جابجایی برای تنوع‌بخشی داده.
2.  Weight Decay (L2 Regularization)

  در بهینه‌ساز (مثل Adam)، اضافه کردن weight_decay=1e-4 برای کاهش پیچیدگی مدل.
  <div>

In [18]:
# یادگیری انتقالی با ResNet18
# ----------------------------
from torchvision.models import resnet18

resnet = resnet18(pretrained=True)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet.fc = nn.Linear(resnet.fc.in_features, 2)
optimizer = optim.Adam(resnet.parameters(), lr=0.0001)

for epoch in range(5):
    resnet.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

resnet.eval()
resnet_preds, resnet_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = resnet(images)
        _, preds = torch.max(outputs, 1)
        resnet_preds.extend(preds.numpy())
        resnet_labels.extend(labels.numpy())

print("\nTransfer Learning (ResNet18):\n", classification_report(resnet_labels, resnet_preds))


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 121MB/s]



Transfer Learning (ResNet18):
               precision    recall  f1-score   support

           0       0.86      0.80      0.83       180
           1       0.90      0.93      0.92       360

    accuracy                           0.89       540
   macro avg       0.88      0.87      0.87       540
weighted avg       0.89      0.89      0.89       540



 <div dir="rtl" style="text-align: right;">

چرا F1-score معیار بهتری است؟

در مسائل پزشکی با داده نامتوازن (مانند بیماری‌های نادر)، Accuracy می‌تواند گمراه‌کننده باشد. فرض کنید فقط 33٪ داده‌ها بیمار باشند. اگر مدلی همه تصاویر را «سالم» تشخیص دهد، ممکن است accuracy بالایی نشان دهد ولی در واقع هیچ‌کار مفیدی انجام نمی‌دهد.

این معیار مناسب است چون:

به نادرستی‌های مثبت کاذب (False Positive) و منفی کاذب (False Negative) حساس است.

در مسائل حساس مانند تشخیص بیماری، تشخیص درست موارد بیمار از اهمیت بالاتری برخوردار است
<div>

 <div dir="rtl" style="text-align: right;">

 چطور عملکرد مدل را عادلانه و قابل اعتماد کنیم؟

 پیشنهادها:

Cross-validation:

استفاده از K-fold (مثلاً 5-fold) برای کاهش تأثیر تصادف در تقسیم داده.

Class-weight یا Oversampling
مقابله با عدم تعادل با:

کلاس‌وزن‌دهی در مدل‌ها (class_weight='balanced')

افزایش داده برای کلاس‌های کم‌تر

Calibrated Probabilities:

مدل باید خروجی احتمال معتبر داشته باشد، نه فقط label. می‌توان از Platt scaling یا Isotonic regression استفاده کرد.

تست انسانی و اعتبارسنجی بالینی:

مدل باید با نظر پزشکان نیز اعتبارسنجی شود تا اعتمادپذیری آن در دنیای واقعی سنجیده شود.

نظارت بر متریک‌های جداگانه برای هر کلاس:

Precision و Recall برای هر کلاس (سالم، بیمار) جداگانه بررسی شود تا مشخص شود مدل در تشخیص بیماران عملکرد خوبی دارد.

In [20]:
import pandas as pd

data = {
    'Model': ['Random Forest', 'Simple CNN', 'Transfer Learning'],
    'Precision': [
        precision_score(y_test, y_pred_rf),
        precision_score(all_labels, all_preds),
        precision_score(resnet_labels, resnet_preds)
    ],
    'Recall': [
        recall_score(y_test, y_pred_rf),
        recall_score(all_labels, all_preds),
        recall_score(resnet_labels, resnet_preds)
    ],
    'Accuracy': [
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(all_labels, all_preds),
        accuracy_score(resnet_labels, resnet_preds)
    ],
    'F1-score': [
        f1_score(y_test, y_pred_rf),
        f1_score(all_labels, all_preds),
        f1_score(resnet_labels, resnet_preds)
    ]
}

summary_df = pd.DataFrame(data)
print("\nModel Evaluation Summary:")
print(summary_df)



Model Evaluation Summary:
               Model  Precision    Recall  Accuracy  F1-score
0      Random Forest   0.833619  0.964286  0.847884  0.894204
1         Simple CNN   0.830000  0.922222  0.822222  0.873684
2  Transfer Learning   0.903226  0.933333  0.888889  0.918033
